In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os
import mmcv
import importlib
import torch
import torchvision.transforms as transforms
from mmcv import Config
from mmcls.datasets.pipelines import Compose
from mmcls.models.builder import build_classifier
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.gridspec as gridspec
from matplotlib import cm
from PIL import Image
import heapq
from mmseg.apis import inference_segmentor, init_segmentor
from mmseg.core.evaluation import get_palette
from mmseg.datasets.builder import build_dataset, build_dataloader, DATASETS
from pytorch_grad_cam.utils.image import show_cam_on_image
import cv2
import pandas
import json
import seaborn as sns

import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    

from scripts import generate_statistics_single
from scripts import visualization_seg_masks
from scripts import generate_statistics
from scripts import generate_cams
from scripts import utils
from scripts.utils.pipeline import get_pipeline_torchvision
from scripts import generate_segs
from scripts import visualize_cam_results
from scripts.utils.BlurDataset import get_blurred_dataset
from scripts.utils import preprocessing, io
from scripts.utils.model import get_wrongly_classified
from scripts.utils.evaluation import compare_original_blurred, get_eval_metrics
from scripts import evaluate_effectiveness

Importing BlurSegments step into Pipeline


In [5]:
configSeg = '../segmentation/ocrnet_hr48_512x512_carparts_noflip/ocrnet_hr48_512x512_carparts_noflip.py'
checkpointSeg = '../segmentation/ocrnet_hr48_512x512_carparts_noflip/latest.pth'

segDataPathOriginal = 'G:/XAI-Vehicle-Classification/preGenData/Segmentations/CompCars_Original/valAllSegs.npz'
segDataPathColor = 'G:/XAI-Vehicle-Classification/preGenData/Segmentations/CompCars_Color/valAllSegs.npz'
segDataPathWeb = 'G:/XAI-Vehicle-Classification/preGenData/Segmentations/CompCars_Web/valAllSegs.npz'
segDataPathStanford = 'G:/XAI-Vehicle-Classification/preGenData/Segmentations/StanfordCars_Original/valAllSegs.npz'

paletteName='Comp_Original_Ocrnet_Carparts_Noflip'

segDataPathOG = 'G:/XAI-Vehicle-Classification/preGenData/Segmentations/CompCars_Original/valAllSegs.npz'
segDataPathNoScaleOG = 'G:/XAI-Vehicle-Classification/preGenData/Segmentations/CompCars_Original/valAllSegsNoScale.npz'
segDataPathNoScaleWeb = 'G:/XAI-Vehicle-Classification/preGenData/Segmentations/CompCars_Web/valAllSegsNoScale.npz'
segDataPathNoScaleStanford = 'G:/XAI-Vehicle-Classification/preGenData/Segmentations/StanfordCars_Original/valAllSegsNoScale.npz'

camDataGradCAM = 'G:/XAI-Vehicle-Classification/preGenData/CompCars_original/ResNet/valAllCAMs.npz'
camDataEigenCAM = 'G:/XAI-Vehicle-Classification/preGenData/CompCars_original/ResNet/valAllCAMsEigen.npz'
camDataEigenGrad = 'G:/XAI-Vehicle-Classification/preGenData/CompCars_original/ResNet/valAllCAMsEigenGrad.npz'
camDataGradCAMGPU = 'G:/XAI-Vehicle-Classification/preGenData/CompCars_original/ResNet/valAllCAMsGPU.npz'
camDataGradCAMPlusPlus = 'G:/XAI-Vehicle-Classification/preGenData/CompCars_original/ResNet/valAllCAMsGrad++.npz'
camDataLayerCAM = 'G:/XAI-Vehicle-Classification/preGenData/CompCars_original/ResNet/valAllCAMsLayer.npz'
camDataXCAM = 'G:/XAI-Vehicle-Classification/preGenData/CompCars_original/ResNet/valAllCAMsXGrad.npz'

imgRootValOriginal = '../data/CompCars_sv_original_split/val'
imgRootValColor = '../data/CompCars_sv_color_split/val'
imgRootValWeb = '../data/CompCars_web_original_split/val'
imgRootValStandford = '../data/StanfordCars_original_split/val'

blurBackgroundImgRootOriginal = '../data/CompCars_sv_original_split/blurred/blurredBackground'
blurBackgrdHoodFBumperImgRootOriginal = '../data/CompCars_sv_original_split/blurred/blurredBackground_Hood_FBumper'
blurFLLightHoodFBumperImgRootOriginal = '../data/CompCars_sv_original_split/blurred/blurredFLLight_Hood_FBumper'
blurFLLightImgRootOriginal = '../data/CompCars_sv_original_split/blurred/blurredFrontLeftLight'

blurWhiteBackgroundImgRootOriginal = '../data/CompCars_sv_original_split/blurred/blurredBackgroundWhite'
blurWhiteBackgrdHoodFBumperImgRootOriginal = '../data/CompCars_sv_original_split/blurred/blurredBackground_Hood_FBumperWhite'
blurWhiteFLLightHoodFBumperImgRootOriginal = '../data/CompCars_sv_original_split/blurred/blurredFLLight_Hood_FBumperWhite'
blurWhiteFLLightImgRootOriginal = '../data/CompCars_sv_original_split/blurred/blurredFrontLeftLightWhite'

blurWhiteBackgroundImgRootWeb = '../data/CompCars_web_original_split/blurred/blurredBackgroundWhite'
blurWhiteBackgrdHoodFBumperImgRootWeb = '../data/CompCars_web_original_split/blurred/blurredBackground_Hood_FBumperWhite'
blurWhiteFLLightHoodFBumperImgRootWeb = '../data/CompCars_web_original_split/blurred/blurredFLLight_Hood_FBumperWhite'
blurWhiteFLLightImgRootWeb = '../data/CompCars_web_original_split/blurred/blurredFrontLeftLightWhite'

blurWhiteBackgroundImgRootStanford = '../data/StanfordCars_original_split/blurred/blurredBackgroundWhite'
blurWhiteBackgrdHoodFBumperImgRootStanford = '../data/StanfordCars_original_split/blurred/blurredBackground_Hood_FBumperWhite'
blurWhiteFLLightHoodFBumperImgRootStanford = '../data/StanfordCars_original_split/blurred/blurredFLLight_Hood_FBumperWhite'
blurWhiteFLLightImgRootStanford = '../data/StanfordCars_original_split/blurred/blurredFrontLeftLightWhite'

annfileValOriginal = '../data/CompCars_sv_original_split/meta/val.txt'
annfileValColor = '../data/CompCars_sv_color_split/meta/val.txt'
annfileValWeb = '../data/CompCars_web_original_split/meta/val.txt'
annfileValStanford = '../data/StanfordCars_original_split/meta/val.txt'

dirResOG = 'G:/results/CompCars_original/ResNet'
dirResCol = 'G:/results/CompCars_Color/ResNet'
dirResWeb = 'G:/results/CompCars_Web/ResNet'

dirSwinBaseOG = 'G:/results/CompCars_original/SwinBase'
dirSwinBaseCol = 'G:/results/CompCars_Color/SwinBase'
dirSwinBaseWeb = 'G:/results/CompCars_Web/SwinBase'

dirSwinSmallOG = 'G:/results/CompCars_original/SwinSmall'
dirSwinSmallCol = 'G:/results/CompCars_Color/SwinSmall'
dirSwinSmallWeb = 'G:/results/CompCars_Web/SwinSmall'

configCAMResOriginal = '../CAMModels/resnet/compCars_Original/resnet50_b128x2_compcars-original-split.py'
checkpointCAMResOriginal = '../CAMModels/resnet/compCars_Original/latest.pth'
configCAMResColor = '../CAMModels/resnet/compCars_Color/resnet50_b128x2_compcars-color-split.py'
checkpointCAMResColor = '../CAMModels/resnet/compCars_Color/latest.pth'
configCAMResWeb = '../CAMModels/resnet/compCars_Web/resnet50_b128x2_compcars-web-original-split.py'
checkpointCAMResWeb = '../CAMModels/resnet/compCars_Web/latest.pth'
configCAMResStanford = '../CAMModels/resnet/stanfordCars_Original/resnet50_b128x2_stanford.py'
checkpointCAMResStanford = '../CAMModels/resnet/stanfordCars_Original/latest.pth'

configCAMSwinBaseOriginal = '../CAMModels/swinBase/compCars_Original/swin-base_b128x2_compcars-original-split.py'
checkpointCAMSwinBaseOriginal = '../CAMModels/swinBase/compCars_Original/latest.pth'
configCAMSwinBaseColor = '../CAMModels/swinBase/compCars_Color/swin-base_b128x2_compcars-color-split.py'
checkpointCAMSwinBaseColor = '../CAMModels/swinBase/compCars_Color/latest.pth'
configCAMSwinBaseWeb = '../CAMModels/swinBase/compCars_Web/swin-base_b128x2_compcars-web-original-split.py'
checkpointCAMSwinBaseWeb = '../CAMModels/swinBase/compCars_Web/latest.pth'
configCAMSwinBaseStanford = '../CAMModels/swinBase/stanfordCars_Original/swin-base_b128x2_stanford.py'
checkpointCAMSwinBaseStanford = '../CAMModels/swinBase/stanfordCars_Original/latest.pth'

configCAMSwinSmallOriginal = '../CAMModels/swinSmall/compCars_Original/swin-small_b128x2_compcars-original-split.py'
checkpointCAMSwinSmallOriginal = '../CAMModels/swinSmall/compCars_Original/latest.pth'
configCAMSwinSmallColor = '../CAMModels/swinSmall/compCars_Color/swin-small_b128x2_compcars-color-split.py'
checkpointCAMSwinSmallColor = '../CAMModels/swinSmall/compCars_Color/latest.pth'
configCAMSwinSmallWeb = '../CAMModels/swinSmall/compCars_Web/swin-small_b128x2_compcars-web-original-split.py'
checkpointCAMSwinSmallWeb = '../CAMModels/swinSmall/compCars_Web/latest.pth'
configCAMSwinSmallStanford = '../CAMModels/swinSmall/stanfordCars_Original/swin-small_b128x2_stanford.py'
checkpointCAMSwinSmallStanford = '../CAMModels/swinSmall/stanfordCars_Original/latest.pth'

eval_data_original_res = 'G:/results/CompCars_original/eval_results_resnet.json'
eval_data_original_swinbase = 'G:/results/CompCars_original/eval_results_swinbase.json'
eval_data_original_swinsmall = 'G:/results/CompCars_original/eval_results_swinsmall.json'

eval_data_color_res = 'G:/results/CompCars_Color/eval_results_resnet.json'
eval_data_color_swinbase = 'G:/results/CompCars_Color/eval_results_swinbase.json'
eval_data_color_swinsmall = 'G:/results/CompCars_Color/eval_results_swinsmall.json'

eval_data_web_res = 'G:/results/CompCars_Web/eval_results_resnet.json'
eval_data_web_swinbase = 'G:/results/CompCars_Web/eval_results_swinbase.json'
eval_data_web_swinsmall = 'G:/results/CompCars_Web/eval_results_swinsmall.json'

eval_data_stanford_res = 'G:/results/StanfordCars_Original/eval_results_resnet.json'
eval_data_stanford_swinbase = 'G:/results/StanfordCars_Original/eval_results_swinbase.json'
eval_data_stanford_swinsmall = 'G:/results/StanfordCars_Original/eval_results_swinsmall.json'

In [ ]:
"""CompCars_Original"""
"""ResNet"""
# Eigencam
baseExcel = os.path.join(dirResOG, 'normalized')
excel = os.path.join(baseExcel,'Data_Full_EigenCAM_ResNet_CompCars_Original_ocrnet_hr48_carparts_noflip_normalized_PropArea_23_08_2022.xlsx')
evaluate_effectiveness.evaluate_blurred_normalized_by_rel_importance(
                                                imgRoot=imgRootValOriginal, classifierConfig=configCAMResOriginal, 
                                                classifierCheckpoint=checkpointCAMResOriginal, annfile=annfileValOriginal,
                                                segData=segDataPathNoScaleOG, segConfig=configSeg,
                                                segCheckpoint=checkpointSeg,  
                                                saveDir='results/CompCars_Original/ResNet/blurRelImpEigen', 
                                                eval_data_original=eval_data_original_res,
                                                importanceScoreFile=excel)

excel = os.path.join(baseExcel,'Data_Full_gradCAM++_ResNet_CompCars_Original_ocrnet_hr48_carparts_noflip_normalized_PropArea_23_08_2022.xlsx')
evaluate_effectiveness.evaluate_blurred_normalized_by_rel_importance(
                                                imgRoot=imgRootValOriginal, classifierConfig=configCAMResOriginal, 
                                                classifierCheckpoint=checkpointCAMResOriginal, annfile=annfileValOriginal,
                                                segData=segDataPathNoScaleOG, segConfig=configSeg,
                                                segCheckpoint=checkpointSeg,  
                                                saveDir='results/CompCars_Original/ResNet/blurRelImpGrad++', 
                                                eval_data_original=eval_data_original_res,
                                                importanceScoreFile=excel)

"""SwinBase"""
# Eigencam
baseExcel = os.path.join(dirSwinBaseOG, 'normalized')
excel = os.path.join(baseExcel,'Data_Full_EigenCAM_SwinBase_CompCars_Original_ocrnet_hr48_carparts_noflip_normalized_PropArea_23_08_2022.xlsx')
evaluate_effectiveness.evaluate_blurred_normalized_by_rel_importance(
                                                imgRoot=imgRootValOriginal, classifierConfig=configCAMSwinBaseOriginal, 
                                                classifierCheckpoint=checkpointCAMSwinBaseOriginal, annfile=annfileValOriginal,
                                                segData=segDataPathNoScaleOG, segConfig=configSeg,
                                                segCheckpoint=checkpointSeg,  
                                                saveDir='results/CompCars_Original/SwinBase/blurRelImpEigen', 
                                                eval_data_original=eval_data_original_swinbase,
                                                importanceScoreFile=excel)

excel = os.path.join(baseExcel,'Data_Full_gradCAM++_SwinBase_CompCars_Original_ocrnet_hr48_carparts_noflip_normalized_PropArea_23_08_2022.xlsx')
evaluate_effectiveness.evaluate_blurred_normalized_by_rel_importance(
                                                imgRoot=imgRootValOriginal, classifierConfig=configCAMSwinBaseOriginal, 
                                                classifierCheckpoint=checkpointCAMSwinBaseOriginal, annfile=annfileValOriginal,
                                                segData=segDataPathNoScaleOG, segConfig=configSeg,
                                                segCheckpoint=checkpointSeg,  
                                                saveDir='results/CompCars_Original/SwinBase/blurRelImpGrad++', 
                                                eval_data_original=eval_data_original_swinbase,
                                                importanceScoreFile=excel)

"""SwinSmall"""
# Eigencam
baseExcel = os.path.join(dirSwinSmallOG, 'normalized')
excel = os.path.join(baseExcel,'Data_Full_EigenCAM_SwinSmall_CompCars_Original_ocrnet_hr48_carparts_noflip_normalized_PropArea_23_08_2022.xlsx')
evaluate_effectiveness.evaluate_blurred_normalized_by_rel_importance(
                                                imgRoot=imgRootValOriginal, classifierConfig=configCAMSwinSmallOriginal, 
                                                classifierCheckpoint=checkpointCAMSwinSmallOriginal, annfile=annfileValOriginal,
                                                segData=segDataPathNoScaleOG, segConfig=configSeg,
                                                segCheckpoint=checkpointSeg,  
                                                saveDir='results/CompCars_Original/SwinSmall/blurRelImpEigen', 
                                                eval_data_original=eval_data_original_swinsmall,
                                                importanceScoreFile=excel)

excel = os.path.join(baseExcel,'Data_Full_gradCAM++_SwinSmall_CompCars_Original_ocrnet_hr48_carparts_noflip_normalized_PropArea_23_08_2022.xlsx')
evaluate_effectiveness.evaluate_blurred_normalized_by_rel_importance(
                                                imgRoot=imgRootValOriginal, classifierConfig=configCAMSwinSmallOriginal, 
                                                classifierCheckpoint=checkpointCAMSwinSmallOriginal, annfile=annfileValOriginal,
                                                segData=segDataPathNoScaleOG, segConfig=configSeg,
                                                segCheckpoint=checkpointSeg,  
                                                saveDir='results/CompCars_Original/SwinSmall/blurRelImpGrad++', 
                                                eval_data_original=eval_data_original_swinsmall,
                                                importanceScoreFile=excel)

In [7]:
"""Blurred Background White"""
# CompCars_Original
evaluate_effectiveness.evaluate_blurred_background(imgRoot=imgRootValOriginal, classifierConfig=configCAMResOriginal, 
                                                classifierCheckpoint=checkpointCAMResOriginal, annfile=annfileValOriginal,
                                                segData=segDataPathNoScaleOG, segConfig=configSeg,
                                                segCheckpoint=checkpointSeg,  
                                                saveDir='results/CompCars_Original/ResNet/backgroundWhite', 
                                                eval_data_original=eval_data_original_res, 
                                                imgRootBlurred=blurWhiteBackgroundImgRootOriginal)
evaluate_effectiveness.evaluate_blurred_background(imgRoot=imgRootValOriginal, classifierConfig=configCAMSwinBaseOriginal, 
                                                classifierCheckpoint=checkpointCAMSwinBaseOriginal, annfile=annfileValOriginal,
                                                segData=segDataPathNoScaleOG, segConfig=configSeg,
                                                segCheckpoint=checkpointSeg,  
                                                saveDir='results/CompCars_Original/SwinBase/backgroundWhite', 
                                                eval_data_original=eval_data_original_swinbase, 
                                                imgRootBlurred=blurWhiteBackgroundImgRootOriginal)
evaluate_effectiveness.evaluate_blurred_background(imgRoot=imgRootValOriginal, classifierConfig=configCAMSwinSmallOriginal, 
                                                classifierCheckpoint=checkpointCAMSwinSmallOriginal, annfile=annfileValOriginal,
                                                segData=segDataPathNoScaleOG, segConfig=configSeg,
                                                segCheckpoint=checkpointSeg,  
                                                saveDir='results/CompCars_Original/SwinSmall/backgroundWhite', 
                                                eval_data_original=eval_data_original_swinsmall, 
                                                imgRootBlurred=blurWhiteBackgroundImgRootOriginal)

# CompCars_Web
evaluate_effectiveness.evaluate_blurred_background(imgRoot=imgRootValWeb, classifierConfig=configCAMResWeb, 
                                                classifierCheckpoint=checkpointCAMResWeb, annfile=annfileValWeb,
                                                segData=segDataPathNoScaleWeb, segConfig=configSeg,
                                                segCheckpoint=checkpointSeg,  
                                                saveDir='results/CompCars_Web/ResNet/backgroundWhite', 
                                                eval_data_original=eval_data_web_res, 
                                                imgRootBlurred=blurWhiteBackgroundImgRootWeb)
evaluate_effectiveness.evaluate_blurred_background(imgRoot=imgRootValWeb, classifierConfig=configCAMSwinBaseWeb, 
                                                classifierCheckpoint=checkpointCAMSwinBaseWeb, annfile=annfileValWeb,
                                                segData=segDataPathNoScaleWeb, segConfig=configSeg,
                                                segCheckpoint=checkpointSeg,  
                                                saveDir='results/CompCars_Web/SwinBase/backgroundWhite', 
                                                eval_data_original=eval_data_web_swinbase, 
                                                imgRootBlurred=blurWhiteBackgroundImgRootWeb)
evaluate_effectiveness.evaluate_blurred_background(imgRoot=imgRootValWeb, classifierConfig=configCAMSwinSmallWeb, 
                                                classifierCheckpoint=checkpointCAMSwinSmallWeb, annfile=annfileValWeb,
                                                segData=segDataPathNoScaleWeb, segConfig=configSeg,
                                                segCheckpoint=checkpointSeg,  
                                                saveDir='results/CompCars_Web/SwinSmall/backgroundWhite', 
                                                eval_data_original=eval_data_web_swinsmall, 
                                                imgRootBlurred=blurWhiteBackgroundImgRootWeb)

# StanfordCars_Original
evaluate_effectiveness.evaluate_blurred_background(imgRoot=imgRootValStandford, classifierConfig=configCAMResStanford, 
                                                classifierCheckpoint=checkpointCAMResStanford, annfile=annfileValStanford,
                                                segData=segDataPathNoScaleStanford, segConfig=configSeg,
                                                segCheckpoint=checkpointSeg,  
                                                saveDir='results/StanfordCars_Original/ResNet/backgroundWhite', 
                                                eval_data_original=eval_data_stanford_res, 
                                                imgRootBlurred=blurWhiteBackgroundImgRootStanford)
evaluate_effectiveness.evaluate_blurred_background(imgRoot=imgRootValStandford, classifierConfig=configCAMSwinBaseStanford, 
                                                classifierCheckpoint=checkpointCAMSwinBaseStanford, annfile=annfileValStanford,
                                                segData=segDataPathNoScaleStanford, segConfig=configSeg,
                                                segCheckpoint=checkpointSeg,  
                                                saveDir='results/StanfordCars_Original/SwinBase/backgroundWhite', 
                                                eval_data_original=eval_data_stanford_swinbase, 
                                                imgRootBlurred=blurWhiteBackgroundImgRootStanford)
evaluate_effectiveness.evaluate_blurred_background(imgRoot=imgRootValStandford, classifierConfig=configCAMSwinSmallStanford, 
                                                classifierCheckpoint=checkpointCAMSwinSmallStanford, annfile=annfileValStanford,
                                                segData=segDataPathNoScaleStanford, segConfig=configSeg,
                                                segCheckpoint=checkpointSeg,  
                                                saveDir='results/StanfordCars_Original/SwinSmall/backgroundWhite', 
                                                eval_data_original=eval_data_stanford_swinsmall, 
                                                imgRootBlurred=blurWhiteBackgroundImgRootStanford)

Evaluating original model vs blurred background.
Evaluating original model vs blurred where segments background are blurred.
Created filtered annotation file at results/CompCars_Original/ResNet/backgroundWhite\annfile_filtered.txt
Using standard dataset on blurred data at directory ../data/CompCars_sv_original_split/blurred/blurredBackgroundWhite
Model already on GPU
Using given evaluation data of original model.
Loading data from Json G:/results/CompCars_original/eval_results_resnet.json
Computing Evaluation Metrics for Blurred Dataset
[>>>>>>>>>>>>>>>>>>>>>>>>] 13333/13333, 242.3 task/s, elapsed: 55s, ETA:     0s
Saving json data to results/CompCars_Original/ResNet/backgroundWhite\eval_results_blurred.json
Add total Change and improvement of original over blurred
Saving evaluation results to results/CompCars_Original/ResNet/backgroundWhite\evalBlurred.xlsx
Removing temporary filtered annotation file results/CompCars_Original/ResNet/backgroundWhite\annfile_filtered.txt
Evaluating orig

In [5]:
get_eval_metrics(configCAMSwinSmallOriginal, checkpointCAMSwinSmallOriginal, blurWhiteBackgroundImgRootOriginal, 
                 annfileValOriginal, saveDir='results/CompCars_Original/SwinSmall/blurBackgrdWhite', 
                 fileName='eval_results_swinsmall')

Evaluating metrics on model.
Created filtered annotation file at ./annfile_filtered.txt
[>>>>>>>>>>>>>>>>>>>>>>>>] 13333/13333, 195.0 task/s, elapsed: 68s, ETA:     0s
Saving json data to results/CompCars_Original/SwinSmall/blurBackgrdWhite\eval_results_swinsmall.json
